In [41]:
import pandas as pd
from pathlib import Path
from pandas.api.types import is_datetime64_any_dtype as is_datetime
import numpy as np
import src.utils as utils
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.pylabtools import figsize
import src.data_processing.data as data
from shapely.geometry import Point

In [2]:
X = data.full_trips['Delay time']
full_trips_sorted = data.full_trips.sort_values(by = ['Train number' , 'Date of departure', 'Time of planned departure', ])
full_trips_sorted

,Unnamed: 0,Date of departure,Train number,Relation,Railway operators,Railway of departure,Time of real arrival,Time of real departure,Time of planned arrival,Time of planned departure,Direction of relation,Name of the stop,Railway of arrival,Date of planned arrival,Date of planned departure,Date of real arrival,Date of real departure,Delay time
13572,13572,2020-09-20,10,ICE,SNCB/NMBS,37,2022-11-02 20:36:30,2022-11-02 20:36:30,2022-11-02 20:29:00,2022-11-02 20:29:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,hergenrath,37,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,450
13575,13575,2020-09-20,10,ICE,SNCB/NMBS,37,2022-11-02 20:47:50,2022-11-02 20:47:50,2022-11-02 20:40:00,2022-11-02 20:40:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,chênée,3,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,470
14842,14842,2020-09-20,10,ICE,SNCB/NMBS,37,2022-11-02 20:48:43,2022-11-02 20:48:43,2022-11-02 20:41:00,2022-11-02 20:41:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,angleur,37,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,463
14845,14845,2020-09-20,10,ICE,SNCB/NMBS,36,2022-11-02 20:51:05,2022-11-02 20:52:54,2022-11-02 20:44:00,2022-11-02 20:46:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,liège-guillemins,37,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,425
14848,14848,2020-09-20,10,ICE,SNCB/NMBS,2,2022-11-02 20:56:54,2022-11-02 20:56:54,2022-11-02 20:51:00,2022-11-02 20:51:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,ans,36,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6358,6358,2020-09-20,19976,IC 19-2,SNCB/NMBS,94,2022-11-02 21:52:36,2022-11-02 21:53:06,2022-11-02 21:50:00,2022-11-02 21:50:00,IC 19-2: TOURNAI -> LILLE FLANDRES,froyennes,94,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,156
23009,23009,2020-09-26,19976,IC 19-2,SNCB/NMBS,94,NaT,2022-11-02 21:45:05,NaT,2022-11-02 21:44:00,IC 19-2: TOURNAI -> LILLE FLANDRES,tournai,NaN,NaN,2020-09-26,NaN,2020-09-26,0
23012,23012,2020-09-26,19976,IC 19-2,SNCB/NMBS,94,2022-11-02 21:49:52,2022-11-02 21:50:13,2022-11-02 21:48:00,2022-11-02 21:48:00,IC 19-2: TOURNAI -> LILLE FLANDRES,froyennes,94,2020-09-26 00:00:00,2020-09-26,2020-09-26,2020-09-26,112
23163,23163,2020-09-26,19979,IC 19-2,SNCB/NMBS,94,2022-11-02 07:16:42,2022-11-02 07:17:50,2022-11-02 07:17:00,2022-11-02 07:17:00,IC 19-2: LILLE FLANDRES -> TOURNAI,froyennes,94,2020-09-26 00:00:00,2020-09-26,2020-09-26,2020-09-26,0


In [14]:
## We cannot just autocorrelate on all trips or on all train numbers, because delay time determined by previous station delay time can only be for that day and that train number, so we take a subset
unique_trainNrs = full_trips_sorted['Train number'].unique()
autocorr_list = []
for day in range(20,27):
    for number in unique_trainNrs[1:500]:
        subset = full_trips_sorted[(full_trips_sorted['Train number'] == number) & (full_trips_sorted['Date of departure'].dt.day == day)]
        if (subset.empty == False & len(subset['Delay time'].unique() >1)):
            autocorr_list.append(subset['Delay time'].autocorr(lag = 1))

autocorr_list


[0.7499253106319347,
 0.8759911218382267,
 0.7590092609454988,
 0.9141473879797798,
 0.8538079623926195,
 0.6462431077625761,
 0.5910637993756989,
 0.6565559269199958,
 0.1929864139970923,
 0.40182362269169775,
 0.7049707944912517,
 0.529623862984512,
 0.37737066653684265,
 0.24323296656289356,
 0.9225441213092802,
 0.9111455860093133,
 0.3456734109914674,
 0.5820780231196807,
 0.10817786632856365,
 0.7008083031923257,
 0.7804893684530112,
 0.8670591291410027,
 0.6032187440837011,
 0.06584375299798537,
 -0.04831294449751429,
 0.8890091561307785,
 0.10578317339359998,
 -0.05555555555555546,
 0.6072588008521863,
 nan,
 0.8993371992665812,
 0.7538962760030534,
 0.29637789379985163,
 0.7493584757571276,
 0.889172718706617,
 0.7184599149510662,
 0.8687899824739698,
 0.8399181047123151,
 0.09042481481758893,
 0.3163398171210637,
 0.7906261898363212,
 0.7327457835031209,
 0.9396813413397679,
 0.9590789344684895,
 0.9414909533847191,
 0.9126334086526171,
 0.802185675826644,
 -0.073913744428370

[0.7499253106319347,
 0.8759911218382267,
 0.7590092609454988,
 0.9141473879797798,
 0.8538079623926195,
 0.6462431077625761,
 0.5910637993756989,
 0.6565559269199958,
 0.1929864139970923,
 0.40182362269169775,
 0.7049707944912517,
 0.529623862984512,
 0.37737066653684265,
 0.24323296656289356,
 0.9225441213092802,
 0.9111455860093133,
 0.3456734109914674,
 0.5820780231196807,
 0.10817786632856365,
 0.7008083031923257,
 0.7804893684530112,
 0.8670591291410027,
 0.6032187440837011,
 0.06584375299798537,
 -0.04831294449751429,
 0.8890091561307785,
 0.10578317339359998,
 -0.05555555555555546,
 0.6072588008521863,
 nan,
 0.8993371992665812,
 0.7538962760030534,
 0.29637789379985163,
 0.7493584757571276,
 0.889172718706617,
 0.7184599149510662,
 0.8687899824739698,
 0.8399181047123151,
 0.09042481481758893,
 0.3163398171210637,
 0.7906261898363212,
 0.7327457835031209,
 0.9396813413397679,
 0.9590789344684895,
 0.9414909533847191,
 0.9126334086526171,
 0.802185675826644,
 -0.073913744428370

In [17]:
routes = {}
for train_number in unique_trainNrs:
    routes[train_number] = data.full_trips[data.full_trips["Train number"] == train_number].sort_values(by= ["Date of real departure",  "Time of real departure"])
correlations = {}
for train_number in unique_trainNrs:
    correlations[train_number] = {}
    for day in routes[train_number]["Date of real departure"].unique():
        correlations[train_number][day] = routes[train_number][routes[train_number]["Date of real departure"] == day]["Delay time"].corr(routes[train_number][routes[train_number]["Date of real departure"] == day]["Delay time"].shift(1))




C:\Users\jonav\anaconda3\lib\site-packages\numpy\lib\function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\jonav\anaconda3\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [57]:
sum = 0
len = 0
for train_number in unique_trainNrs:
    for day in routes[train_number]["Date of real departure"].unique():
        if correlations[train_number][day] != None:
            len +=1
            sum += np.nansum(correlations[train_number][day])

autocorr_mean = sum/len
autocorr_mean

0.32594153223057143

In [ ]:
## We see, on average there is slight form of autocorrelation, meaning that the delay time in the station for that train and for that day is partly determined by the previous delay time: if there was delay time there, the delay time here is positively (increasingly) correlated
## make a scatterplot not correct yet
for train_number in unique_trainNrs:
    for day in routes[train_number]["Date of real departure"].unique():
        plt.scatter(train_number, correlations[train_number][day])